<a href="https://colab.research.google.com/github/manushree635/CV/blob/main/Unetsegementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import os
import torch.nn as nn
import torchvision.transforms.functional as F


In [18]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1),nn.ReLU(inplace=True),nn.Conv2d(out_channels, out_channels, 3, 1, 1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)
    
    

In [3]:
class UNet(nn.Module):
    def __init__(self,in_channels=3,out_channels=2,channels=[64,128,256,512]):
        super(UNet,self).__init__()
        self.downs=nn.ModuleList()
        self.ups=nn.ModuleList()
        self.pool=nn.MaxPool2d(kernel_size=2,stride=2)


        for channel in channels:
            self.downs.append(DoubleConv(in_channels,channel))
            in_channels=channel

        for channel in reversed(channels):
            self.ups.append(nn.ConvTranspose2d(channel*2,channel,kernel_size=2,stride=2))
            self.ups.append(DoubleConv(channel*2,channel))
    
        self.bottleneck=DoubleConv(channels[-1],channels[-1]*2)
        self.final=nn.Conv2d(channels[0],out_channels,kernel_size=1)
        
     
    
    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for i in range(0, len(self.ups), 2):
            x = self.ups[i](x)
            skip_connection = skip_connections[i//2]

            if x.shape != skip_connection.shape:
                x = F.resize(x, size=skip_connection.shape[2:])

            concat = torch.cat((skip_connection, x), dim=1)
            x = self.ups[i+1](concat)

        return self.final(x)

      

      

  
    


In [4]:
def test():
    x = torch.randn((3, 1, 161, 161))
    model = UNet(in_channels=1, out_channels=1)
    preds = model(x)
    print(preds.shape)
    assert preds.shape == x.shape

if __name__ == "__main__":
    test()

torch.Size([3, 1, 161, 161])


In [5]:
!pip install kaggle

In [6]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"manushree31","key":"be65e0be13787e4bca70f31fb9f6ab53"}'}

In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle competitions download -c carvana-image-masking-challenge

 59% 9.00M/15.3M [00:00<00:00, 73.6MB/s]
100% 15.3M/15.3M [00:00<00:00, 74.7MB/s]
  0% 0.00/81.2k [00:00<?, ?B/s]
100% 81.2k/81.2k [00:00<00:00, 81.2MB/s]
 58% 17.0M/29.1M [00:00<00:00, 175MB/s]
100% 29.1M/29.1M [00:00<00:00, 186MB/s]
100% 15.4G/15.4G [06:23<00:00, 57.6MB/s]
100% 15.4G/15.4G [06:23<00:00, 43.3MB/s]
100% 7.76G/7.76G [03:25<00:00, 34.9MB/s]

100% 107k/107k [00:03<00:00, 34.4kB/s]

 99% 793M/804M [00:20<00:00, 61.4MB/s]
100% 804M/804M [00:20<00:00, 40.3MB/s]
  0% 0.00/202k [00:00<?, ?B/s]
100% 202k/202k [00:00<00:00, 65.8MB/s]
 99% 401M/405M [00:07<00:00, 53.4MB/s]
100% 405M/405M [00:07<00:00, 54.0MB/s]


In [9]:
from zipfile import ZipFile
file_name='/content/train.zip'
with ZipFile(file_name,'r') as zip:
  zip.extractall()



from zipfile import ZipFile
file_name='/content/train_masks.zip'
with ZipFile(file_name,'r') as zip:
  zip.extractall()





In [10]:

trainPath = '/content/train'


maskPath = '/content/train_masks'

In [11]:
listt = []
listm = []



for root, directories, files in os.walk(trainPath):
    for name in files:
        listt.append(os.path.join(root, name))
        

for root, directories, files in os.walk(maskPath):
    for name in files:
        listm.append(os.path.join(root, name))

listt.sort()
listm.sort()



In [12]:
!mkdir ./Data1

# Train data folders
!mkdir ./Data1/Train/
!mkdir ./Data1/Train/Images/
!mkdir ./Data1/Train/Masks/


# Validation data folders
!mkdir ./Data1/Val/
!mkdir ./Data1/Val/Images/
!mkdir ./Data1/Val/Masks/

In [13]:
pathImagesTrain = './Data1/Train/Images/'
pathImagesValid = './Data1/Val/Images/'


pathMasksTrain = './Data1/Train/Masks/'
pathMasksValid = './Data1/Val/Masks/'

In [14]:
from distutils.file_util import copy_file
for i in range(len(listt)):
    if i < 48:
        copy_file(listt[i],pathImagesValid )
        copy_file(listm[i],pathMasksValid )

    else:
        copy_file(listt[i],pathImagesTrain )
        copy_file(listm[i],pathMasksTrain )

In [15]:
import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np

In [16]:
import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np

class CarvanaDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        mask_path = os.path.join(self.mask_dir, self.images[index].replace(".jpg", "_mask.gif"))
        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        mask[mask == 255.0] = 1.0

        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]

        return image, mask



In [17]:
!pip install albumentations==0.4.6

     |████████████████████████████████| 122kB 24.7MB/s 
     |████████████████████████████████| 952kB 30.4MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-cp37-none-any.whl size=65175 sha256=d0ae5b0553ab59947264c2e347ff97c568f2e39fc7be6feda4300ebd147cac7a
  Stored in directory: /root/.cache/pip/wheels/c7/f4/89/56d1bee5c421c36c1a951eeb4adcc32fbb82f5344c086efa14
Successfully built albumentations
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [22]:
import torch
import torchvision

from torch.utils.data import DataLoader

def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])

def get_loaders(
    train_dir,
    train_maskdir,
    val_dir,
    val_maskdir,
    batch_size,
    train_transform,
    val_transform,
    num_workers=4,
    pin_memory=True,
):
    train_ds = CarvanaDataset(
        image_dir=train_dir,
        mask_dir=train_maskdir,
        transform=train_transform,
    )

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True,
    )

    val_ds = CarvanaDataset(
        image_dir=val_dir,
        mask_dir=val_maskdir,
        transform=val_transform,
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False,
    )

    return train_loader, val_loader

def check_accuracy(loader, model, device="cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device).unsqueeze(1)
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / (
                (preds + y).sum() + 1e-8
            )

    print(
        f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}"
    )
    print(f"Dice score: {dice_score/len(loader)}")
    model.train()

def save_predictions_as_imgs(
    loader, model, folder="/content/Data1/saved_images", device="cuda"
):
    model.eval()
    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
        torchvision.utils.save_image(
            preds, f"{folder}/pred_{idx}.png"
        )
        torchvision.utils.save_image(y.unsqueeze(1), f"{folder}{idx}.png")

    model.train()

In [23]:
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim


# Hyperparameters etc.
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
NUM_EPOCHS = 3
NUM_WORKERS = 2
IMAGE_HEIGHT = 160  # 1280 originally
IMAGE_WIDTH = 240  # 1918 originally
PIN_MEMORY = True
LOAD_MODEL = False
TRAIN_IMG_DIR = "/content/Data1/Train/Images"
TRAIN_MASK_DIR = "/content/Data1/Train/Masks"
VAL_IMG_DIR = "/content/Data1/Val/Images"
VAL_MASK_DIR="/content/Data1/Val/Masks"

def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())


def main():
    train_transform = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Rotate(limit=35, p=1.0),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

    val_transforms = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

    model = UNet(in_channels=3, out_channels=1).to(DEVICE)
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    train_loader, val_loader = get_loaders(
        TRAIN_IMG_DIR,
        TRAIN_MASK_DIR,
        VAL_IMG_DIR,
        VAL_MASK_DIR,
        BATCH_SIZE,
        train_transform,
        val_transforms,
        NUM_WORKERS,
        PIN_MEMORY,
    )

    if LOAD_MODEL:
        load_checkpoint(torch.load("my_checkpoint.pth.tar"), model)


    check_accuracy(val_loader, model, device=DEVICE)
    scaler = torch.cuda.amp.GradScaler()

    for epoch in range(NUM_EPOCHS):
        train_fn(train_loader, model, optimizer, loss_fn, scaler)

        # save model
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer":optimizer.state_dict(),
        }
        save_checkpoint(checkpoint)

        # check accuracy
        check_accuracy(val_loader, model, device=DEVICE)

        # print some examples to a folder
        save_predictions_as_imgs(
            val_loader, model, folder="/content/Data1/saved_images", device=DEVICE
        )


if __name__ == "__main__":
  main()


  0%|          | 0/315 [00:00<?, ?it/s]

Got 416095/1843200 with acc 22.57
Dice score: 0.368313729763031


100%|██████████| 315/315 [04:14<00:00,  1.24it/s, loss=0.0925]


=> Saving checkpoint
Got 1801695/1843200 with acc 97.75
Dice score: 0.9499224424362183


100%|██████████| 315/315 [04:12<00:00,  1.25it/s, loss=0.0454]


=> Saving checkpoint
Got 1817401/1843200 with acc 98.60
Dice score: 0.9692902565002441


100%|██████████| 315/315 [04:16<00:00,  1.23it/s, loss=0.0349]


=> Saving checkpoint
Got 1820342/1843200 with acc 98.76
Dice score: 0.9728369116783142
